<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 11


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

Создать базовый класс Customer в C#, который будет представлять информацию о клиентах или покупателях. На основе этого класса разработать 2-3 производных класса, демонстрирующих принципы наследования и полиморфизма. В каждом из классов должны быть реализованы новые атрибуты и методы, а также переопределены некоторые методы базового класса для демонстрации полиморфизма.

Требования к базовому классу Customer:

• Атрибуты: Идентификатор клиента (CustomerId), Имя (Name), Электронная почта (Email).

• Методы: o o GetFullName(): метод для получения полного имени клиента. o UpdateEmail(string newEmail): метод для обновления электронной почты клиента. o ViewProfile(): метод для просмотра профиля клиента. Требования к производным классам:

VIPКлиент (VipCustomer): Должен содержать дополнительные атрибуты, такие как Баланс лояльности (LoyaltyPoints). Метод ViewProfile() должен быть переопределен для отображения дополнительной информации о VIPклиенте.
ОбычныйКлиент (RegularCustomer): Должен содержать дополнительные атрибуты, такие как Дата регистрации (RegistrationDate). Метод UpdateEmail() должен быть переопределен для добавления информации о дате последнего обновления электронной почты.
ГрупповойКлиент (GroupCustomer) (если требуется третий класс): Должен содержать дополнительные атрибуты, такие как Название группы (GroupName). Метод GetFullName() должен быть переопределен для отображения названия группы вместо имени клиента.

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) используйте в проекте коллекции, делегаты, события.


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [6]:
using System;
using System.Collections.Generic;
using System.Linq;

// ИНТЕРФЕЙСЫ
public interface ICustomerNotificationService
{
    void SendWelcomeMessage(int customerId);
    void SendPromotion(int customerId, string promotion);
}

public interface ICustomerAuditService
{
    void LogCustomerAction(int customerId, string actionType, string details, DateTime timestamp);
}

public interface IDiscountEligible
{
    decimal GetDiscountPercent();
    bool CanApplyDiscount(decimal purchaseAmount);
}

public interface ILoyaltyEarning
{
    int LoyaltyPoints { get; }
    void AddLoyaltyPoints(int points, string reason);
    bool RedeemLoyaltyPoints(int points);
}

// СЕРВИСЫ
public class EmailCustomerNotificationService : ICustomerNotificationService
{
    public void SendWelcomeMessage(int customerId)
    {
        Console.WriteLine($"   [УВЕДОМЛЕНИЕ] Приветственное письмо отправлено клиенту ID: {customerId}");
    }

    public void SendPromotion(int customerId, string promotion)
    {
        Console.WriteLine($"   [УВЕДОМЛЕНИЕ] Акция для клиента ID {customerId}: {promotion}");
    }
}

public class DatabaseAuditService : ICustomerAuditService
{
    public void LogCustomerAction(int customerId, string actionType, string details, DateTime timestamp)
    {
        Console.WriteLine($"   [АУДИТ] Клиент {customerId}, Действие: {actionType}, Детали: {details}, Время: {timestamp:HH:mm:ss}");
    }
}

// КЛАСС ДЛЯ АРГУМЕНТОВ СОБЫТИЙ
public class CustomerStatusChangedEventArgs : EventArgs
{
    public string OldStatus { get; }
    public string NewStatus { get; }
    public DateTime ChangeTime { get; }
    
    public CustomerStatusChangedEventArgs(string oldStatus, string newStatus)
    {
        OldStatus = oldStatus;
        NewStatus = newStatus;
        ChangeTime = DateTime.Now;
    }
}

// ДЕЛЕГАТЫ
public delegate void CustomerEventHandler(Customer sender, string message);
public delegate void PurchaseEventHandler(Customer customer, decimal amount, string item);
public delegate void StatusChangedEventHandler(object sender, CustomerStatusChangedEventArgs e);
public delegate void DisplayInfoDelegate();

// БАЗОВЫЙ КЛАСС Customer
public class Customer : IDiscountEligible, ILoyaltyEarning
{
    // СОБЫТИЯ
    public event CustomerEventHandler OnProfileUpdated;
    public event PurchaseEventHandler OnPurchaseMade;
    public event StatusChangedEventHandler OnStatusChanged;
    
    // АТРИБУТЫ ИЗ ЗАДАНИЯ
    public int CustomerId { get; protected set; }
    public string Name { get; protected set; }
    public string Email { get; protected set; }
    
    // ДОПОЛНИТЕЛЬНЫЕ АТРИБУТЫ (3-4 шт)
    public string Phone { get; protected set; }
    public DateTime RegistrationDate { get; protected set; }
    public bool IsActive { get; protected set; }
    public string Address { get; protected set; }
    public CustomerType Type { get; protected set; }
    public string SocialMedia { get; protected set; }
    
    // КОЛЛЕКЦИИ
    public List<string> Preferences { get; private set; }
    public Dictionary<string, decimal> PurchaseCategories { get; private set; }
    public Queue<string> RecentActivities { get; private set; }
    public HashSet<string> ViewedProducts { get; private set; }
    
    // ДОПОЛНИТЕЛЬНЫЕ АТРИБУТЫ
    public int LoginCount { get; private set; }
    public DateTime LastLoginDate { get; private set; }
    public decimal TotalPurchasesAmount { get; protected set; }
    public int TotalPurchasesCount { get; protected set; }
    
    // Поле для баллов лояльности (явная реализация интерфейса)
    private int _loyaltyPoints;
    
    // ЗАВИСИМОСТИ
    protected readonly ICustomerNotificationService _notificationService;
    protected readonly ICustomerAuditService _auditService;
    
    // Enum для типа клиента
    public enum CustomerType
    {
        Individual,
        Business,
        Government
    }
    
    // КОНСТРУКТОР
    public Customer(int customerId, string name, string email, CustomerType type,
                   ICustomerNotificationService notificationService, 
                   ICustomerAuditService auditService)
    {
        CustomerId = customerId;
        Name = name;
        Email = email;
        Type = type;
        Phone = "";
        Address = "";
        SocialMedia = "";
        RegistrationDate = DateTime.Now;
        IsActive = true;
        
        // Инициализация коллекций
        Preferences = new List<string>();
        PurchaseCategories = new Dictionary<string, decimal>();
        RecentActivities = new Queue<string>();
        ViewedProducts = new HashSet<string>();
        
        LoginCount = 0;
        LastLoginDate = DateTime.MinValue;
        TotalPurchasesAmount = 0;
        TotalPurchasesCount = 0;
        _loyaltyPoints = 0;
        
        _notificationService = notificationService;
        _auditService = auditService;
        
        _auditService.LogCustomerAction(customerId, "CUSTOMER_CREATED", 
            $"Создан клиент: {name} ({email})", DateTime.Now);
        _notificationService.SendWelcomeMessage(customerId);
        
        RecentActivities.Enqueue($"Создание профиля - {DateTime.Now:HH:mm:ss}");
    }
    
    // ЗАЩИЩЕННЫЕ МЕТОДЫ ДЛЯ ВЫЗОВА СОБЫТИЙ
    protected virtual void RaiseOnProfileUpdated(string message)
    {
        OnProfileUpdated?.Invoke(this, message);
    }
    
    protected virtual void RaiseOnPurchaseMade(decimal amount, string item)
    {
        OnPurchaseMade?.Invoke(this, amount, item);
    }
    
    protected virtual void RaiseOnStatusChanged(string oldStatus, string newStatus)
    {
        OnStatusChanged?.Invoke(this, new CustomerStatusChangedEventArgs(oldStatus, newStatus));
    }
    
    // МЕТОДЫ ИЗ ЗАДАНИЯ
    public virtual string GetFullName()
    {
        return Name;
    }
    
    public virtual void UpdateEmail(string newEmail)
    {
        if (ValidateEmail(newEmail))
        {
            string oldEmail = Email;
            Email = newEmail;
            _auditService.LogCustomerAction(CustomerId, "EMAIL_UPDATED", 
                $"Email изменен: {oldEmail} -> {newEmail}", DateTime.Now);
            RaiseOnProfileUpdated($"Email обновлен: {oldEmail} -> {newEmail}");
            Console.WriteLine($"   [УСПЕХ] Email обновлен: {oldEmail} -> {newEmail}");
        }
        else
        {
            Console.WriteLine("   [ОШИБКА] Некорректный email адрес");
        }
    }
    
    public virtual void ViewProfile()
    {
        Console.WriteLine("\n" + new string('=', 80));
        Console.WriteLine($"ПРОФИЛЬ КЛИЕНТА ID: {CustomerId}");
        Console.WriteLine(new string('=', 80));
        Console.WriteLine($"Имя: {GetFullName()}");
        Console.WriteLine($"Email: {Email}");
        Console.WriteLine($"Телефон: {Phone}");
        Console.WriteLine($"Адрес: {Address}");
        Console.WriteLine($"Тип: {Type}");
        Console.WriteLine($"Соцсети: {SocialMedia}");
        Console.WriteLine($"Дата регистрации: {RegistrationDate:dd.MM.yyyy}");
        Console.WriteLine($"Активен: {(IsActive ? "Да" : "Нет")}");
        Console.WriteLine($"Предпочтения: {Preferences.Count} шт");
        Console.WriteLine($"Логинов: {LoginCount}");
        Console.WriteLine($"Последний вход: {(LastLoginDate == DateTime.MinValue ? "Никогда" : LastLoginDate.ToString("dd.MM.yyyy HH:mm"))}");
        Console.WriteLine($"Покупок: {TotalPurchasesCount} на сумму {TotalPurchasesAmount:C}");
        Console.WriteLine($"Баллы лояльности: {_loyaltyPoints}");
        Console.WriteLine(new string('=', 80));
    }
    
    // ДОПОЛНИТЕЛЬНЫЕ МЕТОДЫ (3-4 шт)
    public virtual void UpdatePhone(string newPhone)
    {
        Phone = newPhone;
        _auditService.LogCustomerAction(CustomerId, "PHONE_UPDATED", 
            $"Телефон обновлен: {newPhone}", DateTime.Now);
        RaiseOnProfileUpdated($"Телефон обновлен: {newPhone}");
        Console.WriteLine($"   [УСПЕХ] Телефон обновлен: {newPhone}");
    }
    
    public virtual void UpdateAddress(string newAddress)
    {
        Address = newAddress;
        _auditService.LogCustomerAction(CustomerId, "ADDRESS_UPDATED", 
            $"Адрес обновлен", DateTime.Now);
        RaiseOnProfileUpdated($"Адрес обновлен: {newAddress}");
        Console.WriteLine($"   [УСПЕХ] Адрес обновлен: {newAddress}");
    }
    
    public virtual void RecordLogin()
    {
        LoginCount++;
        LastLoginDate = DateTime.Now;
        RecentActivities.Enqueue($"Вход в систему - {DateTime.Now:HH:mm:ss}");
        _auditService.LogCustomerAction(CustomerId, "LOGIN_RECORDED", 
            $"Клиент вошел в систему", DateTime.Now);
        Console.WriteLine($"   [УСПЕХ] {Name} вошел в систему. Всего входов: {LoginCount}");
    }
    
    public virtual void AddPurchase(decimal amount, string description = "Покупка", string category = "Общее")
    {
        if (amount > 0)
        {
            TotalPurchasesCount++;
            TotalPurchasesAmount += amount;
            
            // Обновляем категории покупок
            if (PurchaseCategories.ContainsKey(category))
                PurchaseCategories[category] += amount;
            else
                PurchaseCategories[category] = amount;
            
            RecentActivities.Enqueue($"Покупка: {description} - {amount:C}");
            
            _auditService.LogCustomerAction(CustomerId, "PURCHASE_ADDED", 
                $"Покупка: {description} на {amount:C}", DateTime.Now);
            
            // Начисляем баллы лояльности
            int points = (int)(amount / 100);
            ((ILoyaltyEarning)this).AddLoyaltyPoints(points, $"Покупка: {description}");
            
            // Вызываем событие
            RaiseOnPurchaseMade(amount, description);
            
            Console.WriteLine($"   [УСПЕХ] Добавлена покупка #{TotalPurchasesCount}: {description} на сумму {amount:C}");
        }
    }
    
    // ПЕРЕГРУЗКА методов
    public virtual void UpdateEmail(string newEmail, bool sendNotification)
    {
        string oldEmail = Email;
        UpdateEmail(newEmail);
        if (sendNotification)
        {
            _notificationService.SendPromotion(CustomerId, $"Ваш email был изменен на {newEmail}");
            Console.WriteLine($"   [УВЕДОМЛЕНИЕ] Отправлено уведомление на старый email: {oldEmail}");
        }
    }
    
    public void ViewProfile(bool brief)
    {
        if (brief)
        {
            Console.WriteLine($"{GetFullName()} (ID: {CustomerId}, Email: {Email}, Тип: {Type})");
        }
        else
        {
            ViewProfile();
        }
    }
    
    // Метод для делегата DisplayInfoDelegate
    public void DisplayBriefInfo()
    {
        Console.WriteLine($"Клиент: {GetFullName()}, ID: {CustomerId}, Баланс: {TotalPurchasesAmount:C}");
    }
    
    // РЕАЛИЗАЦИЯ ИНТЕРФЕЙСА IDiscountEligible
    public virtual decimal GetDiscountPercent()
    {
        if (TotalPurchasesAmount > 100000) return 15;
        if (TotalPurchasesAmount > 50000) return 10;
        if (TotalPurchasesAmount > 10000) return 5;
        return 0;
    }
    
    public virtual bool CanApplyDiscount(decimal purchaseAmount)
    {
        return purchaseAmount >= 1000 && TotalPurchasesCount > 0;
    }
    
    // РЕАЛИЗАЦИЯ ИНТЕРФЕЙСА ILoyaltyEarning
    int ILoyaltyEarning.LoyaltyPoints => _loyaltyPoints;
    
    void ILoyaltyEarning.AddLoyaltyPoints(int points, string reason)
    {
        if (points > 0)
        {
            int oldPoints = _loyaltyPoints;
            _loyaltyPoints += points;
            _auditService.LogCustomerAction(CustomerId, "LOYALTY_POINTS_ADDED", 
                $"Добавлено {points} баллов. Причина: {reason}", DateTime.Now);
            Console.WriteLine($"   [ЛОЯЛЬНОСТЬ] Добавлено {points} баллов лояльности. Всего: {_loyaltyPoints}");
        }
    }
    
    bool ILoyaltyEarning.RedeemLoyaltyPoints(int points)
    {
        if (points <= _loyaltyPoints && points > 0)
        {
            int oldPoints = _loyaltyPoints;
            _loyaltyPoints -= points;
            _auditService.LogCustomerAction(CustomerId, "LOYALTY_POINTS_REDEEMED", 
                $"Списано {points} баллов", DateTime.Now);
            Console.WriteLine($"   [ЛОЯЛЬНОСТЬ] Списано {points} баллов. Осталось: {_loyaltyPoints}");
            return true;
        }
        Console.WriteLine("   [ОШИБКА] Недостаточно баллов лояльности");
        return false;
    }
    
    // Вспомогательные методы
    protected virtual bool ValidateEmail(string email)
    {
        return !string.IsNullOrEmpty(email) && email.Contains("@") && email.Contains(".");
    }
    
    public int GetCustomerAgeInDays()
    {
        return (DateTime.Now - RegistrationDate).Days;
    }
    
    public bool IsHighlyActive()
    {
        return LoginCount > 10 && GetCustomerAgeInDays() > 7;
    }
    
    public void AddPreference(string preference)
    {
        Preferences.Add(preference);
        Console.WriteLine($"   [УСПЕХ] Добавлено предпочтение: {preference}");
    }
    
    public void AddViewedProduct(string product)
    {
        ViewedProducts.Add(product);
    }
    
    public void ShowRecentActivities(int count = 5)
    {
        Console.WriteLine($"\nПОСЛЕДНИЕ АКТИВНОСТИ КЛИЕНТА {Name}:");
        Console.WriteLine(new string('-', 60));
        int i = 1;
        foreach (var activity in RecentActivities.TakeLast(count))
        {
            Console.WriteLine($"  {i}. {activity}");
            i++;
        }
    }
    
    public void ShowPurchaseCategories()
    {
        Console.WriteLine($"\nКАТЕГОРИИ ПОКУПОК КЛИЕНТА {Name}:");
        Console.WriteLine(new string('-', 60));
        foreach (var category in PurchaseCategories.OrderByDescending(c => c.Value))
        {
            Console.WriteLine($"  - {category.Key}: {category.Value:C}");
        }
    }
    
    // Управление активностью
    public void Deactivate()
    {
        IsActive = false;
        RaiseOnStatusChanged("Активен", "Неактивен");
        _auditService.LogCustomerAction(CustomerId, "CUSTOMER_DEACTIVATED", 
            "Клиент деактивирован", DateTime.Now);
        _notificationService.SendPromotion(CustomerId, "Ваш аккаунт был деактивирован");
        Console.WriteLine($"   [УСПЕХ] Клиент {Name} деактивирован");
    }
    
    public void Activate()
    {
        IsActive = true;
        RaiseOnStatusChanged("Неактивен", "Активен");
        _auditService.LogCustomerAction(CustomerId, "CUSTOMER_ACTIVATED", 
            "Клиент активирован", DateTime.Now);
        _notificationService.SendPromotion(CustomerId, "Ваш аккаунт снова активен!");
        Console.WriteLine($"   [УСПЕХ] Клиент {Name} активирован");
    }
}

// 1. VIPКЛИЕНТ (VipCustomer)
public class VipCustomer : Customer
{
    // ДОПОЛНИТЕЛЬНЫЕ АТРИБУТЫ
    public string VipLevel { get; private set; }
    public decimal VipDiscount { get; private set; }
    public List<string> SpecialOffers { get; private set; }
    public DateTime LastSpecialOfferDate { get; private set; }
    public decimal MonthlySpendingLimit { get; private set; }
    public string PersonalManager { get; private set; }
    public List<string> PurchaseHistory { get; private set; }
    public PriorityQueue<string, int> PriorityTasks { get; private set; }
    
    public VipCustomer(int customerId, string name, string email,
                      ICustomerNotificationService notificationService,
                      ICustomerAuditService auditService,
                      decimal monthlyLimit = 50000,
                      string personalManager = "Не назначен")
        : base(customerId, name, email, CustomerType.Individual, notificationService, auditService)
    {
        MonthlySpendingLimit = monthlyLimit;
        PersonalManager = personalManager;
        SpecialOffers = new List<string>();
        PurchaseHistory = new List<string>();
        PriorityTasks = new PriorityQueue<string, int>();
        LastSpecialOfferDate = DateTime.MinValue;
        UpdateVipLevel();
        
        _auditService.LogCustomerAction(customerId, "VIP_CUSTOMER_CREATED", 
            $"Создан VIP клиент с лимитом {monthlyLimit:C}", DateTime.Now);
    }
    
    // ПЕРЕОПРЕДЕЛЕНИЕ метода ViewProfile()
    public override void ViewProfile()
    {
        Console.WriteLine("\n" + new string('=', 80));
        Console.WriteLine($"ПРОФИЛЬ VIP КЛИЕНТА ID: {CustomerId}");
        Console.WriteLine(new string('=', 80));
        Console.WriteLine($"Имя: {GetFullName()}");
        Console.WriteLine($"Email: {Email}");
        Console.WriteLine($"Уровень VIP: {VipLevel}");
        Console.WriteLine($"Скидка VIP: {VipDiscount}%");
        Console.WriteLine($"Персональный менеджер: {PersonalManager}");
        Console.WriteLine($"Лимит трат в месяц: {MonthlySpendingLimit:C}");
        Console.WriteLine($"Спецпредложения: {SpecialOffers.Count} шт");
        Console.WriteLine($"История покупок: {PurchaseHistory.Count} записей");
        Console.WriteLine($"Приоритетных задач: {PriorityTasks.Count}");
        Console.WriteLine($"Дата регистрации: {RegistrationDate:dd.MM.yyyy}");
        Console.WriteLine($"Баллы лояльности: {((ILoyaltyEarning)this).LoyaltyPoints}");
        Console.WriteLine(new string('=', 80));
    }
    
    public void AddSpecialOffer(string offer)
    {
        SpecialOffers.Add(offer);
        LastSpecialOfferDate = DateTime.Now;
        _auditService.LogCustomerAction(CustomerId, "SPECIAL_OFFER_ADDED", 
            $"Добавлено спецпредложение: {offer}", DateTime.Now);
        RaiseOnProfileUpdated($"Добавлено спецпредложение: {offer}");
        Console.WriteLine($"   [VIP] Добавлено спецпредложение для VIP: {offer}");
    }
    
    public void SetPersonalManager(string managerName)
    {
        PersonalManager = managerName;
        _auditService.LogCustomerAction(CustomerId, "PERSONAL_MANAGER_SET", 
            $"Назначен персональный менеджер: {managerName}", DateTime.Now);
        _notificationService.SendPromotion(CustomerId, $"Вам назначен персональный менеджер: {managerName}");
        Console.WriteLine($"   [VIP] Назначен персональный менеджер: {managerName}");
    }
    
    public void UpdateMonthlyLimit(decimal newLimit)
    {
        MonthlySpendingLimit = newLimit;
        _auditService.LogCustomerAction(CustomerId, "MONTHLY_LIMIT_UPDATED", 
            $"Лимит обновлен: {newLimit:C}", DateTime.Now);
        Console.WriteLine($"   [VIP] Лимит трат обновлен: {newLimit:C}");
    }
    
    public override void AddPurchase(decimal amount, string description = "Покупка", string category = "VIP")
    {
        base.AddPurchase(amount, description, category);
        PurchaseHistory.Add($"{DateTime.Now:dd.MM.yyyy} - {description} - {amount:C}");
        
        int bonusPoints = (int)(amount / 50);
        ((ILoyaltyEarning)this).AddLoyaltyPoints(bonusPoints, "VIP бонус за покупку");
    }
    
    public void AddPriorityTask(string task, int priority)
    {
        PriorityTasks.Enqueue(task, priority);
        Console.WriteLine($"   [ЗАДАЧА] Добавлена задача: {task} с приоритетом {priority}");
    }
    
    public string ProcessNextPriorityTask()
    {
        if (PriorityTasks.Count > 0)
        {
            PriorityTasks.TryDequeue(out string task, out int priority);
            Console.WriteLine($"   [ЗАДАЧА] Обработана задача: {task} (приоритет: {priority})");
            return task;
        }
        return null;
    }
    
    private void UpdateVipLevel()
    {
        if (TotalPurchasesAmount >= 100000)
        {
            VipLevel = "Platinum";
            VipDiscount = 25;
        }
        else if (TotalPurchasesAmount >= 50000)
        {
            VipLevel = "Gold";
            VipDiscount = 20;
        }
        else if (TotalPurchasesAmount >= 10000)
        {
            VipLevel = "Silver";
            VipDiscount = 15;
        }
        else
        {
            VipLevel = "Bronze";
            VipDiscount = 10;
        }
    }
    
    public override decimal GetDiscountPercent()
    {
        decimal baseDiscount = base.GetDiscountPercent();
        return baseDiscount + VipDiscount;
    }
    
    public void ShowSpecialOffers()
    {
        Console.WriteLine($"\nСПЕЦИАЛЬНЫЕ ПРЕДЛОЖЕНИЯ ДЛЯ VIP КЛИЕНТА {Name}:");
        Console.WriteLine(new string('-', 60));
        foreach (var offer in SpecialOffers)
        {
            Console.WriteLine($"  - {offer}");
        }
    }
}

// 2. ОБЫЧНЫЙ КЛИЕНТ (RegularCustomer)
public class RegularCustomer : Customer
{
    // ДОПОЛНИТЕЛЬНЫЕ АТРИБУТЫ
    public DateTime? LastEmailUpdateDate { get; private set; }
    public decimal CreditLimit { get; private set; }
    public bool HasOverdraft { get; private set; }
    public LinkedList<string> Wishlist { get; private set; }
    public decimal MonthlyBudget { get; private set; }
    public decimal CurrentMonthSpending { get; private set; }
    public SortedDictionary<DateTime, string> ScheduledPayments { get; private set; }
    
    public RegularCustomer(int customerId, string name, string email,
                          ICustomerNotificationService notificationService,
                          ICustomerAuditService auditService,
                          decimal creditLimit = 10000,
                          bool hasOverdraft = false,
                          decimal monthlyBudget = 5000)
        : base(customerId, name, email, CustomerType.Individual, notificationService, auditService)
    {
        LastEmailUpdateDate = null;
        CreditLimit = creditLimit;
        HasOverdraft = hasOverdraft;
        MonthlyBudget = monthlyBudget;
        CurrentMonthSpending = 0;
        Wishlist = new LinkedList<string>();
        ScheduledPayments = new SortedDictionary<DateTime, string>();
        
        _auditService.LogCustomerAction(customerId, "REGULAR_CUSTOMER_CREATED", 
            $"Создан обычный клиент с кредитным лимитом {creditLimit:C}", DateTime.Now);
    }
    
    // ПЕРЕОПРЕДЕЛЕНИЕ метода UpdateEmail()
    public override void UpdateEmail(string newEmail)
    {
        base.UpdateEmail(newEmail);
        LastEmailUpdateDate = DateTime.Now;
        Console.WriteLine($"   [ИНФО] Дата обновления email: {LastEmailUpdateDate:dd.MM.yyyy HH:mm:ss}");
        
        if (TotalPurchasesCount > 0)
        {
            AddPurchase(100, "Бонус за обновление email");
        }
    }
    
    public override void ViewProfile()
    {
        Console.WriteLine("\n" + new string('=', 80));
        Console.WriteLine($"ПРОФИЛЬ ОБЫЧНОГО КЛИЕНТА ID: {CustomerId}");
        Console.WriteLine(new string('=', 80));
        Console.WriteLine($"Имя: {GetFullName()}");
        Console.WriteLine($"Email: {Email}");
        Console.WriteLine($"Телефон: {Phone}");
        Console.WriteLine($"Кредитный лимит: {CreditLimit:C}");
        Console.WriteLine($"Овердрафт: {(HasOverdraft ? "Доступен" : "Не доступен")}");
        Console.WriteLine($"Месячный бюджет: {MonthlyBudget:C}");
        Console.WriteLine($"Траты в этом месяце: {CurrentMonthSpending:C}");
        Console.WriteLine($"Список желаний: {Wishlist.Count} товаров");
        Console.WriteLine($"Запланированных платежей: {ScheduledPayments.Count}");
        Console.WriteLine($"Покупок всего: {TotalPurchasesCount} на сумму {TotalPurchasesAmount:C}");
        if (LastEmailUpdateDate.HasValue)
        {
            Console.WriteLine($"Email обновлялся: {LastEmailUpdateDate:dd.MM.yyyy}");
        }
        Console.WriteLine($"Баллы лояльности: {((ILoyaltyEarning)this).LoyaltyPoints}");
        Console.WriteLine(new string('=', 80));
    }
    
    public void AddToWishlist(string item)
    {
        Wishlist.AddLast(item);
        _auditService.LogCustomerAction(CustomerId, "ITEM_ADDED_TO_WISHLIST", 
            $"Добавлено в список желаний: {item}", DateTime.Now);
        Console.WriteLine($"   [СПИСОК] Добавлено в список желаний: {item}");
    }
    
    public void UpdateCreditLimit(decimal newLimit)
    {
        CreditLimit = newLimit;
        _auditService.LogCustomerAction(CustomerId, "CREDIT_LIMIT_UPDATED", 
            $"Кредитный лимит обновлен: {newLimit:C}", DateTime.Now);
        _notificationService.SendPromotion(CustomerId, $"Ваш кредитный лимит увеличен до {newLimit:C}");
        Console.WriteLine($"   [УСПЕХ] Кредитный лимит обновлен: {newLimit:C}");
    }
    
    public void ToggleOverdraft(bool enable)
    {
        HasOverdraft = enable;
        string status = enable ? "включен" : "выключен";
        _auditService.LogCustomerAction(CustomerId, "OVERDRAFT_TOGGLED", 
            $"Овердрафт {status}", DateTime.Now);
        Console.WriteLine($"   [ИНФО] Овердрафт {status}");
    }
    
    public void SetMonthlyBudget(decimal budget)
    {
        MonthlyBudget = budget;
        CurrentMonthSpending = 0;
        _auditService.LogCustomerAction(CustomerId, "MONTHLY_BUDGET_SET", 
            $"Установлен месячный бюджет: {budget:C}", DateTime.Now);
        Console.WriteLine($"   [УСПЕХ] Установлен месячный бюджет: {budget:C}");
    }
    
    public void SchedulePayment(DateTime date, string description)
    {
        ScheduledPayments[date] = description;
        Console.WriteLine($"   [ПЛАТЕЖ] Запланирован платеж на {date:dd.MM.yyyy}: {description}");
    }
    
    public void ShowScheduledPayments()
    {
        Console.WriteLine($"\nЗАПЛАНИРОВАННЫЕ ПЛАТЕЖИ КЛИЕНТА {Name}:");
        Console.WriteLine(new string('-', 60));
        foreach (var payment in ScheduledPayments)
        {
            Console.WriteLine($"  - {payment.Key:dd.MM.yyyy}: {payment.Value}");
        }
    }
    
    public override void AddPurchase(decimal amount, string description = "Покупка", string category = "Обычное")
    {
        if (CurrentMonthSpending + amount > MonthlyBudget && MonthlyBudget > 0)
        {
            Console.WriteLine($"   [ПРЕДУПРЕЖДЕНИЕ] Превышение месячного бюджета! Бюджет: {MonthlyBudget:C}, Траты: {CurrentMonthSpending:C}");
            return;
        }
        
        base.AddPurchase(amount, description, category);
        CurrentMonthSpending += amount;
        
        if (TotalPurchasesCount % 5 == 0)
        {
            ((ILoyaltyEarning)this).AddLoyaltyPoints(50, "Бонус за каждую 5-ю покупку");
        }
    }
    
    public void ShowWishlist()
    {
        Console.WriteLine($"\nСПИСОК ЖЕЛАНИЙ КЛИЕНТА {Name}:");
        Console.WriteLine(new string('-', 60));
        int index = 1;
        foreach (var item in Wishlist)
        {
            Console.WriteLine($"  {index}. {item}");
            index++;
        }
    }
}

// 3. ГРУППОВОЙ КЛИЕНТ (GroupCustomer)
public class GroupCustomer : Customer
{
    // ДОПОЛНИТЕЛЬНЫЙ АТРИБУТ из задания
    public string GroupName { get; private set; }
    
    // ДОПОЛНИТЕЛЬНЫЕ АТРИБУТЫ
    public int MemberCount { get; private set; }
    public string ContactPerson { get; private set; }
    public List<string> Members { get; private set; }
    public decimal GroupDiscount { get; private set; }
    public string Industry { get; private set; }
    public decimal GroupBudget { get; private set; }
    public List<string> GroupProjects { get; private set; }
    public int ActiveProjects { get; private set; }
    public Dictionary<string, List<string>> ProjectTeams { get; private set; }
    
    public GroupCustomer(int customerId, string groupName, string email, string contactPerson,
                        int memberCount, string industry,
                        ICustomerNotificationService notificationService,
                        ICustomerAuditService auditService)
        : base(customerId, groupName, email, CustomerType.Business, notificationService, auditService)
    {
        GroupName = groupName;
        ContactPerson = contactPerson;
        MemberCount = memberCount;
        Industry = industry;
        GroupBudget = 0;
        Members = new List<string>();
        GroupProjects = new List<string>();
        ProjectTeams = new Dictionary<string, List<string>>();
        ActiveProjects = 0;
        GroupDiscount = CalculateGroupDiscount();
        
        _auditService.LogCustomerAction(customerId, "GROUP_CUSTOMER_CREATED", 
            $"Создан групповой клиент: {groupName} ({memberCount} участников)", DateTime.Now);
    }
    
    // ПЕРЕОПРЕДЕЛЕНИЕ метода GetFullName()
    public override string GetFullName()
    {
        return $"Группа: {GroupName}";
    }
    
    public override void ViewProfile()
    {
        Console.WriteLine("\n" + new string('=', 80));
        Console.WriteLine($"ПРОФИЛЬ ГРУППОВОГО КЛИЕНТА ID: {CustomerId}");
        Console.WriteLine(new string('=', 80));
        Console.WriteLine($"Название группы: {GroupName}");
        Console.WriteLine($"Отрасль: {Industry}");
        Console.WriteLine($"Контактное лицо: {ContactPerson}");
        Console.WriteLine($"Email: {Email}");
        Console.WriteLine($"Телефон: {Phone}");
        Console.WriteLine($"Количество участников: {MemberCount}");
        Console.WriteLine($"Зарегистрировано: {Members.Count}");
        Console.WriteLine($"Бюджет группы: {GroupBudget:C}");
        Console.WriteLine($"Скидка группы: {GroupDiscount}%");
        Console.WriteLine($"Проектов: {GroupProjects.Count} (активных: {ActiveProjects})");
        Console.WriteLine($"Команд проектов: {ProjectTeams.Count}");
        Console.WriteLine($"Дата регистрации: {RegistrationDate:dd.MM.yyyy}");
        Console.WriteLine($"Активен: {(IsActive ? "Да" : "Нет")}");
        Console.WriteLine($"Общие покупки: {TotalPurchasesCount} на сумму {TotalPurchasesAmount:C}");
        Console.WriteLine(new string('=', 80));
    }
    
    public void AddMember(string memberName, string position = "Участник")
    {
        if (Members.Count < MemberCount)
        {
            Members.Add($"{memberName} - {position}");
            UpdateGroupDiscount();
            _auditService.LogCustomerAction(CustomerId, "MEMBER_ADDED", 
                $"Добавлен участник: {memberName} ({position})", DateTime.Now);
            RaiseOnProfileUpdated($"Добавлен участник: {memberName}");
            Console.WriteLine($"   [ГРУППА] Добавлен участник: {memberName} ({position})");
        }
        else
        {
            Console.WriteLine($"   [ОШИБКА] Достигнуто максимальное количество участников ({MemberCount})");
        }
    }
    
    public void AddProject(string projectName, decimal budget, bool isActive = true)
    {
        GroupProjects.Add($"{projectName} - {budget:C}");
        GroupBudget += budget;
        if (isActive) ActiveProjects++;
        
        _auditService.LogCustomerAction(CustomerId, "PROJECT_ADDED", 
            $"Добавлен проект: {projectName} с бюджетом {budget:C}", DateTime.Now);
        _notificationService.SendPromotion(CustomerId, $"Создан новый проект: {projectName}");
        
        Console.WriteLine($"   [ПРОЕКТ] Добавлен проект: {projectName} с бюджетом {budget:C}");
    }
    
    public void AddTeamToProject(string projectName, List<string> teamMembers)
    {
        if (!ProjectTeams.ContainsKey(projectName))
        {
            ProjectTeams[projectName] = new List<string>();
        }
        ProjectTeams[projectName].AddRange(teamMembers);
        Console.WriteLine($"   [КОМАНДА] Добавлена команда из {teamMembers.Count} человек в проект {projectName}");
    }
    
    public void UpdateIndustry(string newIndustry)
    {
        Industry = newIndustry;
        _auditService.LogCustomerAction(CustomerId, "INDUSTRY_UPDATED", 
            $"Отрасль обновлена: {newIndustry}", DateTime.Now);
        Console.WriteLine($"   [УСПЕХ] Отрасль обновлена: {newIndustry}");
    }
    
    public void AllocateBudget(decimal amount)
    {
        GroupBudget += amount;
        _auditService.LogCustomerAction(CustomerId, "BUDGET_ALLOCATED", 
            $"Выделен бюджет: {amount:C}", DateTime.Now);
        Console.WriteLine($"   [БЮДЖЕТ] Выделен бюджет: {amount:C}. Общий бюджет: {GroupBudget:C}");
    }
    
    public void SendGroupNotification(string message)
    {
        _auditService.LogCustomerAction(CustomerId, "GROUP_NOTIFICATION_SENT", 
            $"Отправлено уведомление группе: {message}", DateTime.Now);
        RaiseOnProfileUpdated($"Отправлено уведомление группе: {message}");
        Console.WriteLine($"   [УВЕДОМЛЕНИЕ] Уведомление для группы '{GroupName}': {message}");
        Console.WriteLine($"   [ИНФО] Отправлено {Members.Count} участникам");
    }
    
    public void ShowProjectTeams()
    {
        Console.WriteLine($"\nКОМАНДЫ ПРОЕКТОВ ГРУППЫ {GroupName}:");
        Console.WriteLine(new string('-', 60));
        foreach (var project in ProjectTeams)
        {
            Console.WriteLine($"Проект: {project.Key}");
            Console.WriteLine($"  Участники: {string.Join(", ", project.Value)}");
        }
    }
    
    private void UpdateGroupDiscount()
    {
        GroupDiscount = CalculateGroupDiscount();
    }
    
    private decimal CalculateGroupDiscount()
    {
        if (Members.Count >= 50) return 30;
        if (Members.Count >= 20) return 20;
        if (Members.Count >= 10) return 15;
        if (Members.Count >= 5) return 10;
        return 5;
    }
    
    public override decimal GetDiscountPercent()
    {
        decimal baseDiscount = base.GetDiscountPercent();
        return baseDiscount + GroupDiscount;
    }
}

// GENERIC МЕНЕДЖЕР КЛИЕНТОВ
public class CustomerManager<T> where T : Customer
{
    private List<T> _customers = new List<T>();
    private readonly ICustomerNotificationService _notificationService;
    private readonly ICustomerAuditService _auditService;
    
    public CustomerManager(ICustomerNotificationService notificationService,
                          ICustomerAuditService auditService)
    {
        _notificationService = notificationService;
        _auditService = auditService;
    }
    
    public void AddCustomer(T customer)
    {
        _customers.Add(customer);
        _auditService.LogCustomerAction(customer.CustomerId, "CUSTOMER_ADDED_TO_MANAGER", 
            "Клиент добавлен в менеджер", DateTime.Now);
        Console.WriteLine($"   [МЕНЕДЖЕР] Клиент добавлен в менеджер: {customer.GetFullName()}");
    }
    
    public T FindCustomerById(int customerId)
    {
        return _customers.Find(c => c.CustomerId == customerId);
    }
    
    public List<T> FindCustomersByName(string name)
    {
        return _customers.FindAll(c => 
            c.GetFullName().Contains(name, StringComparison.OrdinalIgnoreCase));
    }
    
    public void ShowAllCustomers()
    {
        Console.WriteLine($"\n" + new string('=', 80));
        Console.WriteLine($"ВСЕ КЛИЕНТЫ В МЕНЕДЖЕРЕ ({typeof(T).Name}):");
        Console.WriteLine(new string('=', 80));
        foreach (var customer in _customers)
        {
            Console.WriteLine($"  ID: {customer.CustomerId}, Имя: {customer.GetFullName()}, Email: {customer.Email}, Тип: {customer.Type}");
        }
        Console.WriteLine($"  Всего: {_customers.Count} клиент(ов)");
        Console.WriteLine(new string('=', 80));
    }
    
    public void SendPromotionToAll(string promotion)
    {
        Console.WriteLine($"\nРАССЫЛКА АКЦИИ ДЛЯ ВСЕХ КЛИЕНТОВ:");
        Console.WriteLine(new string('-', 60));
        foreach (var customer in _customers)
        {
            _notificationService.SendPromotion(customer.CustomerId, promotion);
        }
        Console.WriteLine($"  Акция отправлена {_customers.Count} клиентам");
        Console.WriteLine(new string('-', 60));
    }
    
    public void ApplyActionToAll(Action<T> action)
    {
        foreach (var customer in _customers)
        {
            action(customer);
        }
    }
    
    public List<T> GetCustomersByType(Customer.CustomerType type)
    {
        return _customers.Where(c => c.Type == type).ToList();
    }
    
    public int Count => _customers.Count;
}

// ОБРАБОТЧИКИ СОБЫТИЙ
public static class EventHandlers
{
    public static void OnPurchaseMade(Customer customer, decimal amount, string item)
    {
        Console.WriteLine($"   [СОБЫТИЕ] Клиент {customer.Name} совершил покупку {item} на сумму {amount:C}");
    }
    
    public static void OnProfileUpdated(Customer customer, string message)
    {
        Console.WriteLine($"   [СОБЫТИЕ] Профиль клиента {customer.Name} обновлен: {message}");
    }
    
    public static void OnStatusChanged(object sender, CustomerStatusChangedEventArgs e)
    {
        var customer = sender as Customer;
        Console.WriteLine($"   [СОБЫТИЕ] Статус клиента {customer.Name} изменен с '{e.OldStatus}' на '{e.NewStatus}' в {e.ChangeTime:HH:mm:ss}");
    }
}

// ============================================================================
// ОСНОВНОЙ КОД ПРОГРАММЫ
// ============================================================================

Console.WriteLine("\n" + new string('=', 80));
Console.WriteLine("СИСТЕМА УПРАВЛЕНИЯ КЛИЕНТАМИ");
Console.WriteLine(new string('=', 80));

// 1. СОЗДАНИЕ СЕРВИСОВ
Console.WriteLine("\n[ЭТАП 1] СОЗДАНИЕ СЕРВИСОВ");
Console.WriteLine(new string('-', 60));
var notificationService = new EmailCustomerNotificationService();
var auditService = new DatabaseAuditService();
Console.WriteLine("   Сервисы успешно созданы");

// 2. СОЗДАНИЕ КЛИЕНТОВ
Console.WriteLine("\n[ЭТАП 2] СОЗДАНИЕ КЛИЕНТОВ");
Console.WriteLine(new string('-', 60));

var vipCustomer = new VipCustomer(1, "Иван VIP Петров", "ivan.vip@example.com",
    notificationService, auditService, monthlyLimit: 75000, personalManager: "Анна Сидорова");

var regularCustomer = new RegularCustomer(2, "Мария Обычная", "maria@example.com",
    notificationService, auditService, creditLimit: 20000, hasOverdraft: true, monthlyBudget: 10000);

var groupCustomer = new GroupCustomer(3, "ИТ-Департамент ООО 'ТехноПрофи'", 
    "it@techprofile.com", "Сергей Козлов", 15, "IT разработка",
    notificationService, auditService);

Console.WriteLine("   Клиенты успешно созданы");

// 3. ПОДПИСКА НА СОБЫТИЯ
Console.WriteLine("\n[ЭТАП 3] ПОДПИСКА НА СОБЫТИЯ");
Console.WriteLine(new string('-', 60));

vipCustomer.OnPurchaseMade += EventHandlers.OnPurchaseMade;
vipCustomer.OnProfileUpdated += EventHandlers.OnProfileUpdated;
vipCustomer.OnStatusChanged += EventHandlers.OnStatusChanged;

regularCustomer.OnPurchaseMade += EventHandlers.OnPurchaseMade;
regularCustomer.OnProfileUpdated += EventHandlers.OnProfileUpdated;
regularCustomer.OnStatusChanged += EventHandlers.OnStatusChanged;

groupCustomer.OnProfileUpdated += EventHandlers.OnProfileUpdated;
groupCustomer.OnStatusChanged += EventHandlers.OnStatusChanged;

Console.WriteLine("   Подписка на события выполнена");

// 4. РАБОТА С КОЛЛЕКЦИЯМИ
Console.WriteLine("\n[ЭТАП 4] РАБОТА С КОЛЛЕКЦИЯМИ");
Console.WriteLine(new string('-', 60));

// Добавление предпочтений
Console.WriteLine("\nДобавление предпочтений клиентам:");
vipCustomer.AddPreference("Электроника");
vipCustomer.AddPreference("Путешествия");
vipCustomer.AddPreference("Рестораны");

regularCustomer.AddPreference("Книги");
regularCustomer.AddToWishlist("Умные часы");
regularCustomer.AddToWishlist("Наушники");
regularCustomer.AddToWishlist("Ноутбук");

Console.WriteLine("\nДобавление участников в группу:");
groupCustomer.AddMember("Алексей Программист", "Senior Developer");
groupCustomer.AddMember("Мария Дизайнер", "UI/UX Designer");
groupCustomer.AddMember("Сергей Тестировщик", "QA Engineer");
groupCustomer.AddMember("Ольга Менеджер", "Project Manager");

// 5. РАБОТА С ДЕЛЕГАТАМИ
Console.WriteLine("\n[ЭТАП 5] РАБОТА С ДЕЛЕГАТАМИ");
Console.WriteLine(new string('-', 60));

DisplayInfoDelegate displayDelegate = vipCustomer.DisplayBriefInfo;
displayDelegate += regularCustomer.DisplayBriefInfo;
displayDelegate += groupCustomer.DisplayBriefInfo;

Console.WriteLine("Вызов делегата DisplayInfoDelegate:");
displayDelegate();

// 6. ОПЕРАЦИИ С КЛИЕНТАМИ
Console.WriteLine("\n[ЭТАП 6] ОПЕРАЦИИ С КЛИЕНТАМИ");
Console.WriteLine(new string('-', 60));

Console.WriteLine("\nОперации с VIP клиентом:");
vipCustomer.RecordLogin();
vipCustomer.RecordLogin();
vipCustomer.AddPurchase(15000, "Ноутбук премиум", "Электроника");
vipCustomer.AddPurchase(5000, "Смартфон", "Электроника");
vipCustomer.AddSpecialOffer("Бесплатная доставка на год");
vipCustomer.SetPersonalManager("Ольга Смирнова");

// Работа с PriorityQueue
Console.WriteLine("\nРабота с PriorityQueue VIP клиента:");
vipCustomer.AddPriorityTask("Согласование контракта", 1);
vipCustomer.AddPriorityTask("Встреча с менеджером", 2);
vipCustomer.AddPriorityTask("Обновление документов", 3);

Console.WriteLine("\nОперации с обычным клиентом:");
regularCustomer.RecordLogin();
regularCustomer.AddPurchase(3000, "Планшет", "Электроника");
regularCustomer.UpdateCreditLimit(30000);
regularCustomer.SetMonthlyBudget(8000);

// Работа с SortedDictionary
Console.WriteLine("\nРабота с SortedDictionary обычного клиента:");
regularCustomer.SchedulePayment(DateTime.Now.AddDays(7), "Оплата интернета");
regularCustomer.SchedulePayment(DateTime.Now.AddDays(14), "Оплата телефона");

Console.WriteLine("\nОперации с групповым клиентом:");
groupCustomer.RecordLogin();
groupCustomer.AddProject("Разработка CRM системы", 500000, true);
groupCustomer.AddProject("Мобильное приложение", 300000, true);
groupCustomer.AllocateBudget(200000);

// Работа с Dictionary вложенным в List
Console.WriteLine("\nРабота с Dictionary вложенным в List:");
groupCustomer.AddTeamToProject("Разработка CRM системы", 
    new List<string> { "Алексей Программист", "Мария Дизайнер" });
groupCustomer.SendGroupNotification("Собрание в пятницу в 15:00");

// 7. GENERIC МЕНЕДЖЕР КЛИЕНТОВ
Console.WriteLine("\n[ЭТАП 7] GENERIC МЕНЕДЖЕР КЛИЕНТОВ");
Console.WriteLine(new string('-', 60));

var customerManager = new CustomerManager<Customer>(notificationService, auditService);
customerManager.AddCustomer(vipCustomer);
customerManager.AddCustomer(regularCustomer);
customerManager.AddCustomer(groupCustomer);

customerManager.ShowAllCustomers();

// Использование Action делегата
Console.WriteLine("\nПрименение действия ко всем клиентам через Action:");
customerManager.ApplyActionToAll(c => c.RecordLogin());

Console.WriteLine("\nРассылка промоакции всем клиентам:");
customerManager.SendPromotionToAll("Сезонная распродажа!");

// 8. ПОКАЗ ПРОФИЛЕЙ И КОЛЛЕКЦИЙ
Console.WriteLine("\n[ЭТАП 8] ПОКАЗ ПРОФИЛЕЙ И КОЛЛЕКЦИЙ");
Console.WriteLine(new string('-', 60));

Console.WriteLine("\nДетальный профиль VIP клиента:");
vipCustomer.ViewProfile();
vipCustomer.ShowRecentActivities();
vipCustomer.ShowPurchaseCategories();
vipCustomer.ShowSpecialOffers();

Console.WriteLine("\n" + new string('=', 80));

Console.WriteLine("\nДетальный профиль обычного клиента:");
regularCustomer.ViewProfile();
regularCustomer.ShowWishlist();
regularCustomer.ShowScheduledPayments();

Console.WriteLine("\n" + new string('=', 80));

Console.WriteLine("\nДетальный профиль группового клиента:");
groupCustomer.ViewProfile();
groupCustomer.ShowProjectTeams();

// 9. РАБОТА С ПРИОРИТЕТНОЙ ОЧЕРЕДЬЮ
Console.WriteLine("\n[ЭТАП 9] РАБОТА С ПРИОРИТЕТНОЙ ОЧЕРЕДЬЮ");
Console.WriteLine(new string('-', 60));
Console.WriteLine("Обработка приоритетных задач VIP клиента:");
vipCustomer.ProcessNextPriorityTask();
vipCustomer.ProcessNextPriorityTask();

// 10. ДЕМОНСТРАЦИЯ ПОЛИМОРФИЗМА
Console.WriteLine("\n[ЭТАП 10] ДЕМОНСТРАЦИЯ ПОЛИМОРФИЗМА");
Console.WriteLine(new string('-', 60));

// Создание списка клиентов
List<Customer> allCustomersList = new List<Customer> { vipCustomer, regularCustomer, groupCustomer };

Console.WriteLine("\nВызов GetFullName() для всех клиентов:");
foreach (var customer in allCustomersList)
{
    Console.WriteLine($"  {customer.GetType().Name}: {customer.GetFullName()}");
}

Console.WriteLine("\nВызов GetDiscountPercent() для всех клиентов:");
foreach (var customer in allCustomersList)
{
    Console.WriteLine($"  {customer.GetType().Name}: Скидка {customer.GetDiscountPercent()}%");
}

// 11. РАБОТА С ЯВНОЙ РЕАЛИЗАЦИЕЙ ИНТЕРФЕЙСОВ
Console.WriteLine("\n[ЭТАП 11] РАБОТА С ЯВНОЙ РЕАЛИЗАЦИЕЙ ИНТЕРФЕЙСОВ");
Console.WriteLine(new string('-', 60));

ILoyaltyEarning vipLoyalty = vipCustomer;
ILoyaltyEarning regularLoyalty = regularCustomer;

vipLoyalty.AddLoyaltyPoints(500, "Бонус за регистрацию");
regularLoyalty.AddLoyaltyPoints(200, "Приветственный бонус");

Console.WriteLine($"  Баллы VIP клиента: {vipLoyalty.LoyaltyPoints}");
Console.WriteLine($"  Баллы обычного клиента: {regularLoyalty.LoyaltyPoints}");

// 12. ПРОВЕРКА СКИДОК
Console.WriteLine("\n[ЭТАП 12] ПРОВЕРКА СКИДОК НА ПОКУПКИ");
Console.WriteLine(new string('-', 60));

decimal testPurchaseAmount = 1500;
foreach (var customer in allCustomersList)
{
    bool canApply = customer.CanApplyDiscount(testPurchaseAmount);
    decimal discountPercent = customer.GetDiscountPercent();
    decimal discountAmount = testPurchaseAmount * discountPercent / 100;
    
    Console.WriteLine($"\n{customer.GetFullName()}:");
    Console.WriteLine($"  Может применить скидку на {testPurchaseAmount:C}: {(canApply ? "Да" : "Нет")}");
    Console.WriteLine($"  Процент скидки: {discountPercent}%");
    Console.WriteLine($"  Сумма скидки: {discountAmount:C}");
}

// 13. УПРАВЛЕНИЕ АКТИВНОСТЬЮ КЛИЕНТОВ
Console.WriteLine("\n[ЭТАП 13] УПРАВЛЕНИЕ АКТИВНОСТЬЮ КЛИЕНТОВ");
Console.WriteLine(new string('-', 60));

Console.WriteLine("\nДеактивация и активация VIP клиента:");
vipCustomer.Deactivate();
vipCustomer.Activate();

// 14. ФИЛЬТРАЦИЯ И СОРТИРОВКА КОЛЛЕКЦИЙ
Console.WriteLine("\n[ЭТАП 14] ФИЛЬТРАЦИЯ И СОРТИРОВКА КОЛЛЕКЦИЙ");
Console.WriteLine(new string('-', 60));

// Фильтрация клиентов с покупками больше 5000
var highSpenders = allCustomersList.Where(c => c.TotalPurchasesAmount > 5000).ToList();
Console.WriteLine($"\nКлиенты с покупками больше 5000: {highSpenders.Count}");

// Сортировка по сумме покупок
var sortedByPurchases = allCustomersList.OrderByDescending(c => c.TotalPurchasesAmount).ToList();
Console.WriteLine("\nКлиенты отсортированные по сумме покупок:");
foreach (var customer in sortedByPurchases)
{
    Console.WriteLine($"  {customer.GetFullName()}: {customer.TotalPurchasesAmount:C}");
}

// ============================================================================
// ЗАВЕРШЕНИЕ ПРОГРАММЫ
// ============================================================================

Console.WriteLine("\n" + new string('=', 80));
Console.WriteLine("ПРОГРАММА УСПЕШНО ЗАВЕРШЕНА");
Console.WriteLine(new string('=', 80));
Console.WriteLine("\nОБЩАЯ СТАТИСТИКА:");
Console.WriteLine(new string('-', 60));
Console.WriteLine($"  Всего клиентов: {allCustomersList.Count}");
Console.WriteLine($"  Общая сумма покупок: {allCustomersList.Sum(c => c.TotalPurchasesAmount):C}");
Console.WriteLine($"  Общее количество покупок: {allCustomersList.Sum(c => c.TotalPurchasesCount)}");
Console.WriteLine($"  Средний возраст клиентов: {allCustomersList.Average(c => c.GetCustomerAgeInDays()):F1} дней");
Console.WriteLine(new string('-', 60));


СИСТЕМА УПРАВЛЕНИЯ КЛИЕНТАМИ

[ЭТАП 1] СОЗДАНИЕ СЕРВИСОВ
------------------------------------------------------------
   Сервисы успешно созданы

[ЭТАП 2] СОЗДАНИЕ КЛИЕНТОВ
------------------------------------------------------------
   [АУДИТ] Клиент 1, Действие: CUSTOMER_CREATED, Детали: Создан клиент: Иван VIP Петров (ivan.vip@example.com), Время: 19:24:13
   [УВЕДОМЛЕНИЕ] Приветственное письмо отправлено клиенту ID: 1
   [АУДИТ] Клиент 1, Действие: VIP_CUSTOMER_CREATED, Детали: Создан VIP клиент с лимитом ¤75,000.00, Время: 19:24:13
   [АУДИТ] Клиент 2, Действие: CUSTOMER_CREATED, Детали: Создан клиент: Мария Обычная (maria@example.com), Время: 19:24:14
   [УВЕДОМЛЕНИЕ] Приветственное письмо отправлено клиенту ID: 2
   [АУДИТ] Клиент 2, Действие: REGULAR_CUSTOMER_CREATED, Детали: Создан обычный клиент с кредитным лимитом ¤20,000.00, Время: 19:24:14
   [АУДИТ] Клиент 3, Действие: CUSTOMER_CREATED, Детали: Создан клиент: ИТ-Департамент ООО 'ТехноПрофи' (it@techprofile.com), Время: 1